# InternVL3-8B Non-Quantized with Custom Split Model for 4x V100

In [1]:
from pathlib import Path
import random
import math

import numpy as np
import torch
from PIL import Image
from transformers import AutoModel, AutoTokenizer, AutoConfig
import torchvision.transforms as T
from torchvision.transforms.functional import InterpolationMode

# Set random seed for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)
print("✅ Random seed set to 42 for reproducibility")
print(f"🖥️ Available GPUs: {torch.cuda.device_count()}")
for i in range(torch.cuda.device_count()):
    print(f"   GPU {i}: {torch.cuda.get_device_name(i)}")

✅ Random seed set to 42 for reproducibility
🖥️ Available GPUs: 2
   GPU 0: NVIDIA H200
   GPU 1: NVIDIA H200


In [2]:
# Custom split_model function from InternVL3 documentation
# def split_model(model_path):
#     """Create custom device map for InternVL3-8B across multiple GPUs.
    
#     This function intelligently distributes the model layers:
#     - Vision model goes to GPU 0 (uses ~50% capacity)
#     - LLM layers are distributed across all GPUs
#     - Critical components stay on GPU 0 for efficiency
#     """
#     device_map = {}
#     world_size = torch.cuda.device_count()
    
#     # Load config to get model architecture details
#     config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)
#     num_layers = config.llm_config.num_hidden_layers
    
#     # Since the first GPU will be used for ViT, treat it as half a GPU
#     num_layers_per_gpu = math.ceil(num_layers / (world_size - 0.5))
#     num_layers_per_gpu = [num_layers_per_gpu] * world_size
#     num_layers_per_gpu[0] = math.ceil(num_layers_per_gpu[0] * 0.5)
    
#     # Distribute layers across GPUs
#     layer_cnt = 0
#     for i, num_layer in enumerate(num_layers_per_gpu):
#         for j in range(num_layer):
#             if layer_cnt < num_layers:
#                 device_map[f'language_model.model.layers.{layer_cnt}'] = i
#                 layer_cnt += 1
    
#     # Place vision model and critical components on GPU 0
#     device_map['vision_model'] = 0
#     device_map['mlp1'] = 0
#     device_map['language_model.model.embed_tokens'] = 0
#     device_map['language_model.model.norm'] = 0
#     device_map['language_model.model.rotary_emb'] = 0
#     device_map['language_model.lm_head'] = 0
    
#     # Move last layer to GPU 0 for better communication
#     if num_layers > 0:
#         device_map[f'language_model.model.layers.{num_layers - 1}'] = 0
    
#     print(f"📊 Device map created for {num_layers} layers across {world_size} GPUs")
#     print(f"   GPU 0: Vision model + {num_layers_per_gpu[0]} LLM layers + critical components")
#     for i in range(1, world_size):
#         print(f"   GPU {i}: {num_layers_per_gpu[i]} LLM layers")
    
#     return device_map

def split_model(model_path):
    """Create custom device map for InternVL3-8B across multiple GPUs.

    This function intelligently distributes the model layers:
    - Vision model goes to GPU 0 (uses ~50% capacity)
    - LLM layers are distributed across all GPUs
    - Critical components stay on GPU 0 for efficiency
    - FIXED: Last layer is explicitly placed on GPU 0 (not overridden)

    This prevents garbage output (exclamation marks) caused by tensor
    device mismatches during multi-GPU inference.
    """
    device_map = {}
    world_size = torch.cuda.device_count()

    # Load config to get model architecture details
    config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)
    num_layers = config.llm_config.num_hidden_layers

    # Since the first GPU will be used for ViT, treat it as half a GPU
    num_layers_per_gpu = math.ceil(num_layers / (world_size - 0.5))
    num_layers_per_gpu = [num_layers_per_gpu] * world_size
    num_layers_per_gpu[0] = math.ceil(num_layers_per_gpu[0] * 0.5)

    # Distribute layers across GPUs (EXCLUDING the last layer)
    layer_cnt = 0
    for i, num_layer in enumerate(num_layers_per_gpu):
        for _ in range(num_layer):
            if layer_cnt < num_layers - 1:  # FIX: Exclude last layer from loop
                device_map[f'language_model.model.layers.{layer_cnt}'] = i
                layer_cnt += 1

    # Place vision model and critical components on GPU 0
    device_map['vision_model'] = 0
    device_map['mlp1'] = 0
    device_map['language_model.model.embed_tokens'] = 0
    device_map['language_model.model.norm'] = 0
    device_map['language_model.model.rotary_emb'] = 0
    device_map['language_model.lm_head'] = 0

    # FIX: Explicitly place last layer on GPU 0 (critical for preventing garbage output)
    if num_layers > 0:
        device_map[f'language_model.model.layers.{num_layers - 1}'] = 0

    print(f"📊 Device map created for {num_layers} layers across {world_size} GPUs")
    print(f"   GPU 0: Vision model + {num_layers_per_gpu[0]} LLM layers + critical components + LAST LAYER")
    for i in range(1, world_size):
        print(f"   GPU {i}: {num_layers_per_gpu[i]} LLM layers")

    # Debug: Print last few layer assignments
    print("\n🔍 Layer assignment verification:")
    for layer_idx in range(max(0, num_layers - 3), num_layers):
        device = device_map.get(f'language_model.model.layers.{layer_idx}', 'NOT ASSIGNED')
        print(f"   Layer {layer_idx}: GPU {device}")

    return device_map

In [3]:
# Update this path to your local InternVL3-8B model
model_id = "/home/jovyan/nfs_share/models/InternVL3-8B"
# Update this path to your test image
imageName = "/home/jovyan/nfs_share/tod/LMM_POC/evaluation_data/image_008.png"

print("🔧 Loading InternVL3-8B without quantization using custom device mapping...")

# Create custom device map
device_map = split_model(model_id)

# Load model with custom device map (no quantization)
model = AutoModel.from_pretrained(
    model_id,
    # torch_dtype=torch.bfloat16,  # Use bfloat16 for better precision than float16
    torch_dtype=torch.float16,  # Use bfloat16 for better precision than float16
    low_cpu_mem_usage=True,
    device_map=device_map,  # Use custom device mapping
    trust_remote_code=True
).eval()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_id, 
    trust_remote_code=True, 
    use_fast=False
)

print("✅ Model and tokenizer loaded successfully without quantization")
print(f"✅ Model dtype: {model.dtype}")

# Display memory usage per GPU
print("\n💾 GPU Memory Usage After Loading:")
for i in range(torch.cuda.device_count()):
    allocated = torch.cuda.memory_allocated(i) / 1e9
    reserved = torch.cuda.memory_reserved(i) / 1e9
    print(f"   GPU {i}: {allocated:.2f}GB allocated, {reserved:.2f}GB reserved")

🔧 Loading InternVL3-8B without quantization using custom device mapping...
📊 Device map created for 28 layers across 2 GPUs
   GPU 0: Vision model + 10 LLM layers + critical components + LAST LAYER
   GPU 1: 19 LLM layers

🔍 Layer assignment verification:
   Layer 25: GPU 1
   Layer 26: GPU 1
   Layer 27: GPU 0
FlashAttention2 is not installed.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Model and tokenizer loaded successfully without quantization
✅ Model dtype: torch.float16

💾 GPU Memory Usage After Loading:
   GPU 0: 7.96GB allocated, 7.97GB reserved
   GPU 1: 7.92GB allocated, 7.93GB reserved


In [4]:
# Official InternVL3 image preprocessing (from docs)
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
    ])
    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

def load_image(image_file, input_size=448, max_num=12):
    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

# Process image - use bfloat16 to match model dtype
print("🖼️  Processing image...")
# pixel_values = load_image(imageName, max_num=12).to(torch.bfloat16)
pixel_values = load_image(imageName, max_num=12).to(torch.float16).to('cuda:0')

# Move to device where vision model is located (GPU 0 with custom mapping)
vision_device = model.vision_model.device if hasattr(model, 'vision_model') else 'cuda:0'
pixel_values = pixel_values.to(vision_device)

print(f"✅ Image processed: {pixel_values.shape}")
print(f"✅ Number of image tiles: {pixel_values.shape[0]}")
print(f"✅ Pixel values dtype: {pixel_values.dtype}")
print(f"✅ Pixel values on device: {vision_device}")

🖼️  Processing image...
✅ Image processed: torch.Size([7, 3, 448, 448])
✅ Number of image tiles: 7
✅ Pixel values dtype: torch.float16
✅ Pixel values on device: cuda:0


In [5]:
# Visual Question Answering - ask a question about the image
prompt = 'You are an expert document analyzer specializing in bank statement extraction. Extract the transaction data from this Australian bank statement.'
# InternVL3 format: <image>\n + question
formatted_question = f'<image>\n{prompt}'
print(f"❓ Question: {prompt}")

❓ Question: You are an expert document analyzer specializing in bank statement extraction. Extract the transaction data from this Australian bank statement.


In [6]:
# Deterministic generation config with multi-turn support
generation_config = dict(
    max_new_tokens=4000,
    do_sample=False  # Pure greedy decoding for deterministic output
)

# Clear CUDA cache before generation
torch.cuda.empty_cache()

# Generate response using InternVL3 API
print("🤖 Generating response with InternVL3-8B (non-quantized)...")
print(f"\n💾 GPU Memory before generation:")
for i in range(torch.cuda.device_count()):
    allocated = torch.cuda.memory_allocated(i) / 1e9
    reserved = torch.cuda.memory_reserved(i) / 1e9
    print(f"   GPU {i}: {allocated:.2f}GB allocated, {reserved:.2f}GB reserved")

try:
    # Call chat() directly - model is not wrapped
    response, history = model.chat(
        tokenizer, 
        pixel_values, 
        formatted_question, 
        generation_config,
        history=None,
        return_history=True
    )
    
    print("\n✅ Response generated successfully!")
    print("\n" + "="*60)
    print("INTERNVL3 RESPONSE:")
    print("="*60)
    print(response)
    print("="*60)
    
    # Check if response contains only exclamation marks
    if response and all(c in '!' for c in response.strip()):
        print("\n⚠️ WARNING: Response contains only exclamation marks!")
        print("This indicates a device mismatch issue. Consider:")
        print("1. Using the 8-bit quantized version instead")
        print("2. Adjusting the device mapping strategy")
        print("3. Ensuring all model components are properly placed")
    
except Exception as e:
    print(f"\n❌ Error during inference: {e}")
    print(f"Error type: {type(e).__name__}")
    import traceback
    traceback.print_exc()
finally:
    # Display final memory usage
    print(f"\n💾 GPU Memory after generation:")
    for i in range(torch.cuda.device_count()):
        allocated = torch.cuda.memory_allocated(i) / 1e9
        reserved = torch.cuda.memory_reserved(i) / 1e9
        print(f"   GPU {i}: {allocated:.2f}GB allocated, {reserved:.2f}GB reserved")
    
    # Clean up GPU memory
    torch.cuda.empty_cache()

🤖 Generating response with InternVL3-8B (non-quantized)...

💾 GPU Memory before generation:
   GPU 0: 7.97GB allocated, 7.99GB reserved
   GPU 1: 7.92GB allocated, 7.93GB reserved


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.



✅ Response generated successfully!

INTERNVL3 RESPONSE:
![](https://i!image!i!com!i!n!i!g!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i!u!i!n!i!g!i

In [7]:
# Optional: Debug device placement
print("🔍 Model Component Device Placement:")
print(f"   Vision model: {model.vision_model.device if hasattr(model, 'vision_model') else 'N/A'}")
print(f"   Language model embeddings: {model.language_model.model.embed_tokens.weight.device if hasattr(model, 'language_model') else 'N/A'}")

# MLP1 is a Sequential module - check its first layer
if hasattr(model, 'mlp1'):
    if isinstance(model.mlp1, torch.nn.Sequential):
        # Get device from first layer in the Sequential
        first_layer = list(model.mlp1.children())[0]
        if hasattr(first_layer, 'weight'):
            print(f"   MLP1 (first layer): {first_layer.weight.device}")
        else:
            print(f"   MLP1: Sequential module on cuda:0")
    else:
        print(f"   MLP1: {model.mlp1.weight.device if hasattr(model.mlp1, 'weight') else 'N/A'}")
else:
    print("   MLP1: N/A")

# Check layer distribution
if hasattr(model, 'language_model') and hasattr(model.language_model, 'model'):
    print("\n📊 LLM Layer Distribution:")
    layer_devices = {}
    for i, layer in enumerate(model.language_model.model.layers):
        device = str(layer.self_attn.q_proj.weight.device)
        if device not in layer_devices:
            layer_devices[device] = []
        layer_devices[device].append(i)
    
    for device, layers in sorted(layer_devices.items()):
        print(f"   {device}: layers {layers[0]}-{layers[-1]} ({len(layers)} layers)")

🔍 Model Component Device Placement:
   Vision model: cuda:0
   Language model embeddings: cuda:0
   MLP1 (first layer): cuda:0

📊 LLM Layer Distribution:
   cuda:0: layers 0-27 (11 layers)
   cuda:1: layers 10-26 (17 layers)


In [8]:
# Optional: Save response to file
try:
    output_path = Path("internvl3_8b_custom_split_output.txt")
    
    with output_path.open("w", encoding="utf-8") as text_file:
        text_file.write(response)
    
    print(f"✅ Response saved to: {output_path}")
    print(f"📄 File size: {output_path.stat().st_size} bytes")
    
    # Quick content check
    if output_path.stat().st_size < 100:
        print("⚠️ Warning: Response file is very small, might indicate an issue")
    
except NameError:
    print("❌ Error: 'response' variable not defined.")
    print("💡 Please run the generation cell first.")
    
except Exception as e:
    print(f"❌ Error saving file: {e}")

✅ Response saved to: internvl3_8b_custom_split_output.txt
📄 File size: 2681 bytes


## Multi-Turn Conversation Example

InternVL3 supports multi-turn conversations using the history parameter:

In [9]:
# Second turn conversation (uses history from first turn)
try:
    follow_up_question = "Can you provide the total amount from the document?"
    print(f"❓ Follow-up question: {follow_up_question}")
    
    # Use history from previous turn
    response2, history = model.chat(
        tokenizer,
        None,  # No new image for follow-up
        follow_up_question,
        generation_config,
        history=history,  # Pass previous history
        return_history=True
    )
    
    print("\n" + "="*60)
    print("FOLLOW-UP RESPONSE:")
    print("="*60)
    print(response2)
    print("="*60)
    
except NameError:
    print("❌ Error: 'history' variable not defined.")
    print("💡 Please run the first generation cell to create conversation history.")
except Exception as e:
    print(f"❌ Error during follow-up: {e}")

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


❓ Follow-up question: Can you provide the total amount from the document?

FOLLOW-UP RESPONSE:
![](!!image!i!!n!i!g!!i!u!i!n!i!g!!!i!u!!n!i!g!!!!i!u!!!!!!n!!i!!!!!!!g!!!!!!i!!!!!u!!!!!!n!!i!!!!!!!g!!!!!!i!!!!!u!!!!!!n!!i!!!!!!!g!!!!!!i!!!!!u!!!!!!n!!i!!!!!!!g!!!!!!i!!!!!u!!!!!!n!!i!!!!!!!g!!!!!!i!!!!!u!!!!!!n!!i!!!!!!!g!!!!!!i!!!!!u!!!!!!n!!i!!!!!!!g!!!!!!i!!!!!u!!!!!!n!!i!!!!!!!g!!!!!!i!!!!!u!!!!!!n!!i!!!!!!!g!!
